In [ ]:
!unzip "/content/drive/MyDrive/MajorAssignmentData/Dataset 1.zip"

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.models import mobilenet_v2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip uninstall torch torchvision -y
!pip install torch torchvision

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


data_dir = '/content/Dataset 1/Colorectal Cancer '


classes = ['MUS', 'NORM', 'STR']


images = []
labels = []


for class_name in classes:
    class_path = os.path.join(data_dir, class_name)
    for filename in os.listdir(class_path):
        if filename.endswith('.tif'):
            image_path = os.path.join(class_path, filename)
            image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
            images.append(image)
            labels.append(class_name)


X = np.array(images)
y = np.array(labels)


label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)


X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)




In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms
from torchvision.models import mobilenet_v2
from sklearn.metrics import accuracy_score

In [ ]:
batch_size = 32
# Define data transformations
data_transform = transforms.Compose([
    transforms.ToPILImage(),  # Ensure input is a PIL image
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Apply transformations to your data
X_train_transformed = torch.stack([data_transform(x) for x in X_train])
X_val_transformed = torch.stack([data_transform(x) for x in X_val])

# Convert data to PyTorch tensors
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# Create DataLoader for training and validation sets
train_dataset = TensorDataset(X_train_transformed, y_train_tensor)
val_dataset = TensorDataset(X_val_transformed, y_val_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:


# Define hyperparameters

learning_rate = 0.001
epochs = 10

# Define MobileNetV2 model
model = mobilenet_v2(pretrained=False, num_classes=len(classes))

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validation
    model.eval()
    val_predictions = []
    val_true_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            val_predictions.extend(predicted.numpy())
            val_true_labels.extend(labels.numpy())

    val_accuracy = accuracy_score(val_true_labels, val_predictions)
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader)}, Validation Accuracy: {val_accuracy}")

# Save the trained model



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 1/10, Loss: 0.5848494052886963, Validation Accuracy: 0.7483333333333333
Epoch 2/10, Loss: 0.3610184720158577, Validation Accuracy: 0.6466666666666666
Epoch 3/10, Loss: 0.2848620306700468, Validation Accuracy: 0.7083333333333334
Epoch 4/10, Loss: 0.23549387770394484, Validation Accuracy: 0.715
Epoch 5/10, Loss: 0.19064868637671073, Validation Accuracy: 0.6533333333333333
Epoch 6/10, Loss: 0.1906558569148183, Validation Accuracy: 0.7583333333333333
Epoch 7/10, Loss: 0.14599895712609093, Validation Accuracy: 0.87
Epoch 8/10, Loss: 0.1510388885997236, Validation Accuracy: 0.5816666666666667
Epoch 9/10, Loss: 0.12946304828549424, Validation Accuracy: 0.7483333333333333
Epoch 10/10, Loss: 0.11464312985228996, Validation Accuracy: 0.8333333333333334


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms
from torchvision.models import resnet18
from torchvision import datasets
from sklearn.metrics import accuracy_score

# Define hyperparameters
batch_size = 32
learning_rate = 0.001
epochs = 10

# Define data transformations
data_transform = transforms.Compose([
    transforms.ToPILImage(),  # Ensure input is a PIL image
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Apply transformations to your data
X_train_transformed = torch.stack([data_transform(x) for x in X_train])
X_val_transformed = torch.stack([data_transform(x) for x in X_val])

# Convert data to PyTorch tensors
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# Create DataLoader for training and validation sets
train_dataset = TensorDataset(X_train_transformed, y_train_tensor)
val_dataset = TensorDataset(X_val_transformed, y_val_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define ResNet-18 model with pre-trained weights
model = resnet18(pretrained=True)
# Modify the final fully connected layer for the number of classes in your dataset
model.fc = nn.Linear(model.fc.in_features, len(classes))

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validation
    model.eval()
    val_predictions = []
    val_true_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            val_predictions.extend(predicted.numpy())
            val_true_labels.extend(labels.numpy())

    val_accuracy = accuracy_score(val_true_labels, val_predictions)
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader)}, Validation Accuracy: {val_accuracy}")

# Save the trained model
torch.save(model.state_dict(), '/content/trained_resnet18.pth')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 93.4MB/s]


Epoch 1/10, Loss: 0.2480997323182722, Validation Accuracy: 0.7516666666666667
Epoch 2/10, Loss: 0.11646742386432986, Validation Accuracy: 0.66
Epoch 3/10, Loss: 0.09539381618766735, Validation Accuracy: 0.7633333333333333
Epoch 4/10, Loss: 0.08707807161612437, Validation Accuracy: 0.915
Epoch 5/10, Loss: 0.061380552371653414, Validation Accuracy: 0.99
Epoch 6/10, Loss: 0.06140756510392142, Validation Accuracy: 0.8983333333333333
Epoch 7/10, Loss: 0.025333555562732122, Validation Accuracy: 0.9816666666666667
Epoch 8/10, Loss: 0.07151410000020406, Validation Accuracy: 0.9666666666666667
Epoch 9/10, Loss: 0.04084833827374192, Validation Accuracy: 0.9133333333333333
Epoch 10/10, Loss: 0.023079741673718672, Validation Accuracy: 0.98
